In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import os
import sys
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..\..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)

In [228]:
from funcs import SVM
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split
import itertools
from sklearn.model_selection import KFold

In [229]:
df = pd.read_csv('data/Letters_Q_O.csv')

train_df, test_df = train_test_split(df, test_size=0.2, random_state=1939671)

In [230]:
def process_df(df):
    X = np.array(df.drop(['letter'], axis=1))
    y_letter = np.array(df[['letter']])

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)

    y = []
    for i in y_letter:
        if i[0] == 'Q':
            y.append(-1)
        else:
            y.append(1)
    y = np.array(y)
    return X, y

In [81]:
X_train = X[:int(len(X)*0.8)]
X_test = X[int(len(X)*0.8):]
y_train = y[:int(len(X)*0.8)]
y_test = y[int(len(X)*0.8):]

In [ ]:
grid_C = [5, 10, 15, 20, 25, 30, 40, 50]
grid_gamma = [1, 2, 3, 4]
grid_kernel = ['polynomial', 'rbf']

iterables = [grid_C, grid_gamma, grid_kernel]
min_loss = 1000
k_fold = 5

kf5 = KFold(n_splits=k_fold, shuffle=False)

list_ = [['C', 'Gamma', 'Kernel', 'Validation Loss']]

for t in itertools.product(*iterables):
    
    val_loss = 0
    
    C = t[0]
    gamma = t[1]
    kernel = t[2]
    
    print('C: ', C)
    print('gamma: ', gamma)
    print('kernel: ', kernel)
    
    for train_index, test_index in kf5.split(train_df):
        
        X_, y_ = process_df(train_df.iloc[train_index])
        X_val, y_val = process_df(train_df.iloc[test_index])

        svm = SVM(X_, y_, C=C, gamma=gamma, kernel=kernel)
        svm.fit(tol=1e-5, fix_intercept=False)
        
        val_loss += svm.pred(X_val, y_val)
        
    val_loss = val_loss/k_fold
    print('Accuracy: ', val_loss)
    print('=======================')
    print('')
    
    list_.append([C, gamma, kernel, val_loss])

C:  5
gamma:  1
kernel:  polynomial
Accuracy:  0.9653371945066036

C:  5
gamma:  1
kernel:  rbf
Accuracy:  0.8810758632793647

C:  5
gamma:  2
kernel:  polynomial
Accuracy:  0.9063884866833399

C:  5
gamma:  2
kernel:  rbf
Accuracy:  0.8632705892676935

C:  5
gamma:  3
kernel:  polynomial
Accuracy:  0.5243955947523146

C:  5
gamma:  3
kernel:  rbf
Accuracy:  0.8614014303891887

C:  5
gamma:  4
kernel:  polynomial
Accuracy:  0.5243955947523146

C:  5
gamma:  4
kernel:  rbf


In [ ]:
import csv

with open("res.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(list_)